In [2]:
import numpy as np
import OpenEXR
import pywt

from Imath import PixelType
from scipy.sparse import bsr_array, csr_array, save_npz, load_npz


IMG_WIDTH, IMG_HEIGHT = 512, 256
ENV_WIDTH, ENV_HEIGHT = 512, 256

In [2]:
def load_exr(filename):
  exr = OpenEXR.InputFile(filename)
  dw = exr.header()['dataWindow']
  width = dw.max.x - dw.min.x + 1
  height = dw.max.y - dw.min.y + 1

  img = np.zeros((height, width, 3), dtype=np.float32)
  for i, c in enumerate('RGB'):
    buffer = exr.channel(c, PixelType(OpenEXR.FLOAT))
    img[:, :, i] = np.frombuffer(buffer, dtype=np.float32).reshape(height, width)

  exr.close()
  return img


def save_exr(filename, img):
  height, width, _ = img.shape
  header = OpenEXR.Header(width, height)
  exr = OpenEXR.OutputFile(filename, header)
  
  r, g, b = np.split(img, 3, axis=-1)
  exr.writePixels({'R': r.tobytes(),
                   'G': g.tobytes(),
                   'B': b.tobytes()})
  exr.close()

In [ ]:
T = np.zeros((IMG_WIDTH * IMG_HEIGHT, ENV_WIDTH * ENV_HEIGHT // 2), dtype=np.float32)

for idx, (i, j) in enumerate(np.ndindex(ENV_HEIGHT // 2, ENV_WIDTH)):
  img = load_exr(f'../out/renderer/{i:03d},{j:03}.exr')
  # Con un img colorata togliere il [:, :, 0] che seleziona solo 1 canale, ma devo selezionarli tutti e 3
  coeffs = pywt.wavedec2(img[:, :, 0], 'haar')
  coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)
  T[:, idx] = coeff_arr.ravel()

In [15]:
asdasd = load_exr(f'../out/renderer/{i:03d},{j:03}.exr')
asdasd.shape

(256, 512, 3)

In [5]:
img = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.float32)
print(img.shape)
coeffs = pywt.wavedec2(img[:, :, 0], "haar")
_, coeff_slices = pywt.coeffs_to_array(coeffs)

(256, 512, 3)


In [ ]:
for idx, (i, j) in enumerate(np.ndindex(ENV_HEIGHT // 2, ENV_WIDTH)):
    if i == 8 and j == 75:
        print(f"{idx}")

In [ ]:
T = np.zeros((IMG_WIDTH * IMG_HEIGHT, ENV_WIDTH * ENV_HEIGHT // 2), dtype=np.float32)

In [ ]:
img = load_exr('../out/renderer/064,256.exr')
coeffs = pywt.wavedec2(img, 'haar', axes=(0, 1))
coeff_arr, _ = pywt.coeffs_to_array(coeffs, axes=(0, 1))
save_exr('../img.exr', img)
save_exr('../coeff.exr', coeff_arr)

In [4]:
np.count_nonzero(coeff_arr) / coeff_arr.size

1.0

In [5]:
(coeff_arr.size - np.isclose(coeff_arr, 0, atol=5e-4).sum()) / coeff_arr.size

np.float64(0.05841827392578125)

In [ ]:
EPS = 5e-4
T[np.isclose(T, 0, atol=EPS)] = 0

In [ ]:
T_bsr = bsr_array(T, dtype=np.float32)
T_csr = csr_array(T, dtype=np.float32)

In [ ]:
print(T.nbytes * 1e-9, (T_bsr.data.nbytes + T_bsr.indptr.nbytes + T_bsr.indices.nbytes) * 1e-9,
      (T_csr.data.nbytes + T_csr.indptr.nbytes + T_csr.indices.nbytes) * 1e-9)

In [ ]:
save_npz('../T_bsr.npz', T_bsr)

In [ ]:
save_npz('../T_csr.npz', T_csr)

### SECONDO SCRIPT

In [ ]:
T_csr = load_npz("../T_csr.npz")
T_bsr = load_npz("../T_bsr.npz")

In [ ]:
# Runna questa
L = load_exr('../resources/meadow_2_90deg.exr')
L = L[:ENV_HEIGHT // 2, :, :].reshape(-1, 3)

B = T_bsr @ L * 1e-4 # Fare una costante 1e-4

B = B.reshape(IMG_HEIGHT, IMG_WIDTH, 3)
coeffs = pywt.array_to_coeffs(B, coeff_slices, output_format='wavedec2')
B = pywt.waverec2(coeffs, 'haar', axes=(0, 1))

save_exr('../out.exr', B)

In [7]:
# Non Runnare se runni quella sopra
L = load_exr('../resources/meadow_2_90deg.exr')# .reshape(-1, 3)

L = L[:ENV_HEIGHT // 2, :, :].reshape(-1, 3)

B = T @ L * 1e-4

save_exr('../out.exr', B.reshape(IMG_HEIGHT, IMG_WIDTH, 3))

In [8]:
stroke_img = load_exr(f'../resources/stroke.exr').reshape(-1, 3)

in_stroke = (1, 0, 0)
out_stroke = (0, 1, 0)

X_in = np.all(stroke_img == in_stroke, axis=-1)
X_out = np.all(stroke_img == out_stroke, axis=-1)

In [9]:
T_in = np.mean(T[X_in, :], axis=0)
T_out = np.mean(T[X_out, :], axis=0)

delta = T_out[:, None] * L - T_in[:, None] * L

In [ ]:
# Per salvare l'albedo io devo caricare la scena blender
# Andare su "View layer" e attivare Diffuse:Color e salvare il render (che sarebbe l'albedo)
p = load_exr('../resources/albedo.exr').reshape(-1, 3)

L_avg = np.mean(L, axis=0)
p_avg = np.mean(p, axis=0)

L_f = np.any(delta > 0.8 * L_avg * p_avg, axis=-1)
L_b = ~L_f

In [ ]:
# Me la salvo intera non a meta
env = L.copy()
# env = load_exr('../resources/meadow_2_90deg.exr').reshape(-1, 3)
env[L_f, :] = (1, 0, 0)
ENV_HEIGHT, ENV_WIDTH = 512, 256
# ENV_HEIGHT, ENV_WIDTH = 256, 256
save_exr('../env.exr', env.reshape(ENV_HEIGHT // 2, ENV_WIDTH, 3))

mask = np.zeros_like(L, dtype=np.float32)
mask[L_f, :] = 1.0
mask_img = np.zeros((ENV_HEIGHT, ENV_WIDTH, 3), dtype=np.float32)
mask_img[:ENV_HEIGHT // 2, :, :] = mask.reshape(ENV_HEIGHT // 2, ENV_WIDTH, 3)

In [13]:
ENV_HEIGHT = ENV_HEIGHT // 2
f = np.zeros_like(L, dtype=np.float32)
b = np.zeros_like(L, dtype=np.float32)
mask = np.zeros_like(L, dtype=np.float32)

f[L_f, :] = L[L_f, :]
b[L_b, :] = L[L_b, :]

f_edit = np.roll(f.reshape(ENV_HEIGHT, ENV_WIDTH, 3), -ENV_WIDTH // 2, axis=1).reshape(-1, 3)
L_edit = np.where(f_edit > 0, f_edit, b)

mask[L_f, :] = 1.0
mask[f_edit > 0] = 1.0

In [14]:
B_edit = T @ L_edit * 1e-4

save_exr('../env.exr', L_edit.reshape(ENV_HEIGHT, ENV_WIDTH, 3))
save_exr('../mask.exr', mask.reshape(ENV_HEIGHT, ENV_WIDTH, 3))
save_exr('../out.exr', B_edit.reshape(IMG_HEIGHT, IMG_WIDTH, 3))